In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import copy
import re
import numpy as np
import json
# input the sst json path
sst_path = "sst_json.json"
# input the LLM result path
result_path = "./movie"

In [29]:
def clean(str):
    str = str.lower()
    str =  re.sub(r"[\'\n]", '', str)
    str = re.split(r"\d+\. ",str)[1:]
    temp = []
    for _ in str:
        t = _.find('-')
        if t > -1:
            temp.append(_[:t])
        else:
            temp.append(_)
    str = temp
    temp = []
    for _ in str:
        t = _.find('\"')
        if t > -1:
            fix = re.findall(r'"([^"]*)"', _)
            if len(fix) == 0:
                temp.append(_.replace('\"','').strip(' '))
            else:
                temp.append(fix[0].strip(' '))
        else:
            temp.append(_.strip(' '))
    str = temp
    return str

def get_clean_rec_list(result_csv, n=100, k=20):
    final_dict = {}
    for i in range(n):
        try:
            clean_rec_list = clean(result_csv["Result"][i])
            final_dict[result_csv["name"][i]] = clean_rec_list
        except:
            continue
    return final_dict

def simplified_list(songs_list):
    simplified_list = []
    for songs in songs_list:
        songs = re.sub(r"\([^)]*\)", "", songs)
        simplified_list.append(re.sub(r"[ ]", "", songs))
    return simplified_list

def calc_serp_ms(x, y):
    temp = 0
    if len(y) == 0:
        return 0
    for i, item_x in enumerate(x):
        for j, item_y in enumerate(y):
            if item_x == item_y:
                temp = temp + len(x) - i + 1    
    return temp * 0.5 / ((len(y) + 1) * len(y))

def calc_prag(x, y):
    temp = 0
    sum = 0
    if len(y) == 0 or len(x) == 0 :
        return 0
    if len(x) == 1:
        if x == y:
            return 1
        else: 
            return 0
    for i, item_x1 in enumerate(x):
        for j, item_x2 in enumerate(x):
            if i >= j:
                continue
            id1 = -1
            id2 = -1
            for k, item_y in enumerate(y):
                if item_y == item_x1:
                    id1 = k
                if item_y == item_x2:
                    id2 = k
            sum = sum + 1
            if id1 == -1:
                continue
            if id2 == -1:
                temp = temp + 1
            if id1 < id2:
                temp = temp + 1
    return temp / sum


def calc_metric_at_k(list1, list2, top_k=20, metric = "iou"):
    if metric == "iou":
        x = set(list1[:top_k])
        y = set(list2[:top_k])
        metric_result = len(x & y) / len(x | y)
    elif metric == "serp_ms":
        x = list1[:top_k]
        y = list2[:top_k]
        metric_result = calc_serp_ms(x, y)
    elif metric == "prag":
        x = list1[:top_k]
        y = list2[:top_k]
        metric_result = calc_prag(x, y)
    return metric_result


def calc_mean_metric_k(iou_dict, top_k=20):
    mean_list = []
    for i in range(1,top_k + 1):
        mean_list.append(np.mean(np.array(iou_dict[i])))
    return mean_list

def get_metric_with_neutral(compared_path, neutral_path = "neutral.csv", n=100, top_k=20, metric = "iou"):
    compare_result_csv = pd.read_csv(compared_path)
    neutral_result_csv = pd.read_csv(neutral_path)
    compare_clean_rec_list= get_clean_rec_list(compare_result_csv, n=n, k=top_k)
    neutral_clean_rec_list= get_clean_rec_list(neutral_result_csv, n=n, k=top_k)
    compare_neutral_metric = {i : [] for i in range(1, top_k + 1)}
    for artist in compare_clean_rec_list.keys():
        print(artist)
        compare_list = compare_clean_rec_list[artist]
        neutral_list = neutral_clean_rec_list[artist]
        compare_simp_list = simplified_list(compare_list)
        neutral_simp_list = simplified_list(neutral_list)
        print(compare_simp_list)
        print(neutral_simp_list)
        for k in range(1,top_k+1):
            compare_neutral_metric[k].append(calc_metric_at_k(compare_simp_list, neutral_simp_list,k, metric=metric))
    return compare_neutral_metric

In [30]:
def return_min_max_delta_std(sst_path, result_path, keys = ['age', 'country', 'gender', 'continent', 'occupation', 'race', 'religion',  'physics'], metric = "iou", K = 20):
    f = open(sst_path)
    data = json.load(f)
    max_list = []
    min_list = []
    delta_list = []
    std_list = []
    for i in range(len(keys)):
        sst_metric_list = []
        for result in data[keys[i]]:
            #result = f"{keys[i]}/{result.replace(' ','_')}"
            result = f"{keys[i]}/{result}"
            sst_metric_list.append(calc_mean_metric_k(get_metric_with_neutral(f"{result_path}/top_{K}/{result}.csv",f"{result_path}/top_{K}/neutral/neutral.csv", n=5,top_k=K,metric = metric))[-1])
        sst_metric_list = np.array(sst_metric_list)
        max_list.append(sst_metric_list.max())
        min_list.append(sst_metric_list.min())
        delta_list.append(sst_metric_list.max() - sst_metric_list.min())
        std_list.append(sst_metric_list.std())
    return max_list, min_list, delta_list, std_list

In [31]:
result_dict = {}
for metric in ["iou", "serp_ms", "prag"]:
    max_temp, min_temp, delta_temp, std_temp= return_min_max_delta_std(sst_path, result_path, metric=metric, K = 25)
    result_dict[metric] = {}
    result_dict[metric]["max"] = max_temp
    result_dict[metric]["min"] = min_temp
    result_dict[metric]["SNSR"] = delta_temp
    result_dict[metric]["SNSV"] = std_temp
cont_list = []
keys = ['age', 'country', 'gender', 'continent', 'occupation', 'race', 'religion',  'physics']
for metric in result_dict.keys():
    temp_dict = result_dict[metric]
    for method in temp_dict.keys():
        result_dict_temp = {}
        result_dict_temp["name"] = method + "_" + metric
        for i in range(len(keys)):
            result_dict_temp[keys[i]] = temp_dict[method][i]
        cont_list.append(result_dict_temp)
df = pd.DataFrame(cont_list, columns=["name"] + keys)
df.to_csv("result.csv")

Mark Cendrowski
['thebiglebowski', 'thehangover', 'bridesmaids', '21jumpstreet', 'anchorman:thelegendofronburgundy', 'superbad', 'the40', 'knockedup', 'stepbrothers', 'tropicthunder', 'pineappleexpress', 'theotherguys', 'ted', 'werethemillers', '22jumpstreet', 'theheat', 'neighbors', 'theinterview', 'spy', 'sausageparty', 'badmoms', 'gamenight', 'blockers', 'theniceguys', 'booksmart']
['thebigbangtheory', 'mike&molly', 'mom', 'twoandahalfmen', 'thekingofqueens', 'everybodylovesraymond', 'frasier', 'seinfeld', 'friends', 'curbyourenthusiasm', 'parksandrecreation', 'theoffice', 'modernfamily', 'arresteddevelopment', 'community', 'brooklynnine', 'newgirl', 'superstore', 'thegoodplace', 'schittscreek', 'tedlasso', 'abbottelementary', 'onlymurdersinthebuilding', 'hacks', 'theafterparty']
Pamela Fryman
['theparenttrap', 'freakyfriday', 'meangirls', 'bringiton', '13goingon', 'theprincessdiaries', 'confessionsofateenagedramaqueen', 'thelizziemcguiremovie', 'shestheman', 'easya', 'meangirls', '